In [1]:
s = [16, 42, 28, 3, 26, 0, 31, 46, 27, 14, 49, 62, 37, 56, 23, 6, 40,
     48, 53, 8, 20, 25, 33, 1, 2, 63, 15, 34, 55, 21, 39, 57, 54, 45,
     47, 13, 7, 44, 61, 9, 60, 32, 22, 29, 52, 19, 12, 50, 5, 51, 11,
     18, 59, 41, 36, 30, 17, 38, 10, 4, 58, 43, 35, 24]

p = [24, 5, 15, 23, 14, 32, 19, 18, 26, 17, 6, 12, 34, 9, 8, 20, 28, 0,
     2, 21, 29, 11, 33, 22, 30, 31, 1, 25, 3, 35, 16, 13, 27, 7, 10, 4]

########################################
# S-box function
########################################
def sbox(x):
    return s[x]

########################################
# P-box function
########################################
def pbox(x):
    y = 0
    # For each bit to be shuffled
    for i in range(len(p)):
        # If the original bit position
        # is a 1, then make the result
        # bit position have a 1
        if (x & (1 << i)) != 0:
            y = y ^ (1 << p[i])
    return y

########################################
# Takes 36-bit to six 6-bit values
# and vice-versa
########################################
def demux(x):
    y = []
    for i in range(6):
        y.append((x >> (i * 6)) & 0x3f)
    
    return y

def mux(x):
    y = 0
    for i in range(6):
        y = y ^ (x[i] << (i * 6))
    
    return y

########################################
# Key mixing
########################################
def mix(p, k):
    v = []
    key = demux(k)
    for i in range(6):
        v.append(p[i] ^ key[i])
    
    return v

########################################
# Round function
########################################
def round(p, k):
    u = []
    
    # Calculate the S-boxes
    for x in demux(p):
        u.append(sbox(x))
    
    # Run through the P-box
    v = demux(pbox(mux(u)))
    
    # XOR in the key
    w = mix(v, k)
    
    # Glue back together, return
    return mux(w)

# Listing 6-1
Helper functions for performing linear cryptanalysis on the EASY1 cipher.

In [2]:
#######################################
# Calculate the reverse S-box
#######################################
s2 = []
for i in range(0, len(s)):
    s2.append(0)
for i in range(0, len(s)):
    s2[s[i]] = i
    
def asbox(x):
    return s2[x]

#######################################
# Reverse P-box
#######################################
def apbox(x):
    y = 0
    for i in range(len(p)):
        if (x & (1 << p[i])) != 0:
            y = y ^ (1 << i)
    return y

#######################################
# Extract a bit from the argument
#######################################
def grab(x, pos):
    return (x >> (pos)) & 1

# Listing 6-2
Generating known plaintext-ciphertext pairs for use in EASY1 linear cryptanalysis

In [3]:
# Set the number of known plaintext-ciphertext pairs
# 1800 is sufficient for our simple expression
numplaintexts = 1800

# Here we set a simple key
key = mux([0x12, 0x12, 0x12, 0x12, 0x12, 0x12])

# The encryption function, with no key schedule
def encrypt(p, rounds):
    x = p
    for i in range(rounds):
        x = round(x, key)
    return x

# Import the random package
import random

# Use a fixed seed, so that we get reproducibility
random.seed(12345)

# Create lists for the plaintext and ciphertext
plaintext = []
ciphertext = []

# Generate the texts
for i in range(0, numplaintexts):
    # Generate a random plaintext
    r = random.randint(0, 2**36)
    plaintext.append(r)
    
    # Also store the corresponding ciphertext
    c = encrypt(r, 3)
    ciphertext.append(c)

# Listing 6-3
Linear cryptanalysis on EASY1.
Set all of the counts to zero, and then, for each possible subkey, take every
plaintext-ciphertext pair and see if the linear expression is true, and if so,
increment the count of the key.

In [8]:
# Best deviation so far
maxdev = -1

# Best deviation's index
maxk = -1

# Which S-box we are working with
koffset = 6

# Initialise all of the counts to zero
ssize = 2 ** 6
count = [0 for i in range(ssize)]

# Brute force the subkeys
for k1 in range(ssize):
    # Calculate target partial subkey
    k = k1 << koffset
    
    # For each plaintext-ciphertext pair
    for j in range(0, len(plaintext)):
        
        # Get the pair
        pt = plaintext[j]
        ct = ciphertext[j]
        
        # Undo the last mixing layer with
        # target partial subkey
        v = apbox(ct)
        v = demux(v)
        v = mix(v, k)
        
        # Go backwards through last S-box
        u = mux([asbox(v[0]), asbox(v[1]), asbox(v[2]),
                 asbox(v[3]), asbox(v[4]), asbox(v[5])])
        
        # If the linear expression holds, increment
        # the appropriate count
        if grab(pt, 6) ^ grab(pt, 7) ^ grab(pt, 8) ^ \
        grab(pt, 9) ^ grab(pt, 10) ^ grab(u, 8) ^ \
        grab(u, 9) == 0:
            count[k1] = count[k1] + 1
        
    # If this was the best so far, then mark it
    if abs(count[k1] - len(plaintext) / 2) >= maxdev:
        maxdev = abs(count[k1] - len(plaintext) / 2)
        maxk = k
    
    # Uncomment the following line to see the progress
    print(k1, count[k1])

0 874
1 1085
2 885
3 966
4 956
5 888
6 895
7 921
8 813
9 882
10 882
11 845
12 955
13 836
14 918
15 880
16 865
17 907
18 895
19 839
20 889
21 883
22 868
23 864
24 888
25 905
26 954
27 900
28 955
29 915
30 973
31 947
32 887
33 968
34 922
35 891
36 932
37 929
38 898
39 972
40 838
41 920
42 919
43 871
44 844
45 839
46 839
47 886
48 816
49 903
50 863
51 875
52 909
53 845
54 867
55 875
56 911
57 930
58 939
59 898
60 996
61 837
62 985
63 868


# Listing 6-4
Print results of previous linear cryptanalysis of EASY1.
Note that the $rjust$ function takes the string it is applied to and pads it up to the
given length (the first argument) by inserting the pad character (the second argument)
on the left until it is the correct length.

In [28]:
# Prints out its argument in binary
def binary(x):
    if x != 0:
        y = binary(x >> 1) + str(x % 2)
        if y == "":
            return "0"
        else:
            return y
    else:
        return ""

print("guess:", binary(maxk >> koffset))
print(maxk >> koffset)
print("  deviation: ", maxdev/float(len(plaintext)))

print("real:", binary((apbox(key) >> koffset) & 0x3f))
print((apbox(key) >> koffset) & 0x3f)

guess: 1
1
  deviation:  0.10277777777777777
real: 1
1


In [37]:
len(binary(key))

35